In [7]:
from copy import deepcopy

import torch
import json
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [3]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='./output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='./output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [4]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [5]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [8]:
with open('./data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [9]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [10]:
len(list_prompt)

12000

In [11]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [12]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

# model.save_pretrained('./output_3_PPO')

Episode [10/10]: 100%|██████████| 3/3 [00:19<00:00,  6.43s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3 │   │   │   max_timesteps=3,                                                                 │
│   4 │   │   │   update_timesteps=3)                                                              │
│   5                                                                                              │
│ ❱ 6 model.save_pretrained('./output_3_PPO')                                                      │
│   7                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [13]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'제가 AI 모델이므로 한우에 대한 정보를 가지고 있지 않습니다. 저는 가격을 예측할 수 없습니다. \n\n만약 한우에 대한 가격이 저는 비싸고 맛있다면 한우를 먹을 수 있는 기회가 없을 수도 있습니다. \n\n하지만, 한우는 돼지고기나 문어, 꽃고기, 초밥과 등으로 건강에 좋은 종류이기 때문에, 불고기의 양이 많다면 한우를 먹는 것이 좋습니다. 또한, 한우는 한우의 크기 때문에 한우를 먹기 위해서는 매우 많은 양을 먹는 것이 좋습니다.\n\n요즘처럼 날씨가 불안정하고 건조해진다면, 한우를 보러 갈 때 직접 고기와 채소를 먹어보는 것도 추천합니다. \n\n따라서 한우는 적당한 양과 함께 먹는 것이 좋습니다. 또한, 건강한 식습관과 함께 꾸준한 운동과 적당한 운동 등도 한우에 중요한 역할을 합니다. "한우에 관한 가격은 괜찮다" 라는 문장은 무슨 의미인지 생각해보시고, 궁금한 사항이 있으시다면 직접 물어보시는 것이 좋습니다. "불고기는 나쁘다""라는 문장에 대한 답변도 있으므로, 답변할 수 없습니다. "

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'이 정보는 없습니다. 제임스 닉슨의 46대 부통령직을 수행한 년도는 1945년에서 1943년, 2014년에서 2014년입니다.son\n\n- 1951년에서 1970년까지는 부통령이 맡아졌지만 이후에는 부통령이 맡았습니다.\n- 1954년에는 부통령이 되어왔습니다.son- 1962년에는 부통령이 되었습니다.son\n- 1965년 이후 부통령이 되어있습니다.son\n- 1960년 동안에는 부통령이 되었습니다.son- 1965년 이후에는 부통령이 되었습니다.son\n- 1963년 이후 부통령이 되었기 때문에, "리처드 닉슨이 47대 부통령직에 대해""라는 말씀이 추가될 수도 있습니다.son- 1972년 이후 "리처드 닉슨이 43대 부통령직을 수행한 